# SCIP で二次割当問題を解く

参考: https://scmopt.github.io/opt100/34qap.html

問題インスタンスは、一階層上のディレクトリで次を実行:
```
sh 3_Quadratic_Assignment/get_qap_data.sh
```
(上記リンクに貼られたリンクはアクセスできなかった(2024/11/11)ため、他のサイトへのアクセスで代用している)

In [1]:
from util import read_qap
from pyscipopt import Model, quicksum

folder = '../data/qap/problem/'

## 一つ目の定式化

In [5]:
n, f, d = read_qap(folder +"tai20a.dat")
V = list(range(n))
model = Model("qap")
x = {}
for i in V:
    for j in V:
        x[i, j] = model.addVar(vtype="B", name=f"x[{i},{j}]")

for j in V:
    model.addCons(quicksum(x[i, j] for i in V) == 1)
for i in V:
    model.addCons(quicksum(x[i, j] for j in V) == 1)

# pyscipopt では二次の目的関数を，二次の制約に帰着して表現する
z = model.addVar(name='objective')
model.addCons(
    z >= quicksum(
        f[i, j] * d[k, ell] * x[i, k] * x[j, ell]
        for i in V
        for j in V
        for k in V
        for ell in V
    )
)

model.setObjective(
    z,
    sense='minimize'
)
# 600秒に設定するとなぜか動かなかった (2024.11.11)
model.setParam('limits/time', 300.0)
model.optimize()

presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 40 upgd conss, 0 impls, 40 clqs
(round 2, exhaustive) 0 del vars, 0 del conss, 2142 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 41 upgd conss, 0 impls, 40 clqs
   (3.2s) probing: 1000/1522 (65.7%) - 0 fixings, 0 aggregations, 1499334 implications, 0 bound changes
   (3.2s) probing: 1001/1522 (65.8%) - 0 fixings, 0 aggregations, 1500835 implications, 0 bound changes
   (3.2s) probing aborted: 1000/1000 successive useless probings
   (3.2s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (3.2s) no symmetry present (symcode time: 0.01)
presolving (3 rounds: 3 fast, 3 medium, 3 exhaustive):
 1 deleted vars, 1 deleted constraints, 2142 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 340 implications, 841 cliques
presolved problem has 1862 variables (400 bin, 0 int, 1462 impl, 0

## 二つ目の定式化

In [6]:
n, f, d = read_qap(folder +"tai20a.dat")
V = list(range(n))

M ={}
for i in V:
    for k in V:
        sum_ = 0
        for j in V: 
            for ell in V:
                sum_ += f[i,j]*d[k,ell]
        M[i,k] = sum_

model = Model("qap-linear")
x, w = {}, {}
for i in V:
    for j in V:
        w[i, j] = model.addVar(vtype="C", name=f"w[{i},{j}]")
        x[i, j] = model.addVar(vtype="B", name=f"x[{i},{j}]")

for j in V:
    model.addCons(quicksum(x[i, j] for i in V) == 1)
for i in V:
    model.addCons(quicksum(x[i, j] for j in V) == 1)

for i in V:
    for k in V:
        model.addCons(
            M[i,k]*(x[i,k]-1) + quicksum(
                f[i,j]*d[k,ell]*x[j,ell]for j in V for ell in V
            ) <= w[i,k]
        )

# pyscipopt では二次の目的関数を，二次の制約に帰着して表現する
z = model.addVar(name='objective')
model.addCons(
    z >= quicksum(
        w[i, k]
        for i in V
        for k in V
    )
)

model.setObjective(
    z,
    sense='minimize'
)
model.setParam('limits/time', 300.0)
model.optimize()

presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 400 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 40 clqs
(round 2, fast)       1 del vars, 1 del conss, 0 add conss, 400 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 40 clqs
(round 3, exhaustive) 1 del vars, 1 del conss, 0 add conss, 400 chg bounds, 0 chg sides, 0 chg coeffs, 40 upgd conss, 0 impls, 40 clqs
   (0.1s) probing: 51/400 (12.8%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.1s) probing aborted: 50/50 successive totally useless probings
   (0.2s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.2s) no symmetry present (symcode time: 0.01)
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 1 deleted constraints, 0 added constraints, 400 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 40 cliques
presolved problem has 800 variables (400 bin,

0 | 445 |  21k|  2 | 496 |2031 | 0.000000e+00 | 7.475240e+05 |    Inf | unknown
  296s|  4900 |  4901 |  1103k| 182.9 |   455M | 114 | 800 | 916 | 441 |  21k|  1 | 503 |2031 | 0.000000e+00 | 7.475240e+05 |    Inf | unknown
  297s|  5000 |  5001 |  1110k| 181.0 |   458M | 114 | 800 | 925 | 444 |  21k|  1 | 515 |2032 | 0.000000e+00 | 7.475240e+05 |    Inf | unknown
  298s|  5100 |  5101 |  1115k| 178.8 |   461M | 114 | 800 | 927 | 452 |  21k|  1 | 519 |2032 | 0.000000e+00 | 7.475240e+05 |    Inf | unknown
  300s|  5200 |  5201 |  1121k| 177.0 |   464M | 117 | 800 | 935 | 446 |  21k|  2 | 528 |2033 | 0.000000e+00 | 7.475240e+05 |    Inf | unknown

SCIP Status        : solving was interrupted [time limit reached]
Solving Time (sec) : 300.00
Solving Nodes      : 5226 (total of 6275 nodes in 2 runs)
Primal Bound       : +7.47524000000000e+05 (618 solutions)
Dual Bound         : +0.00000000000000e+00
Gap                : infinite
